In [266]:
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [267]:


data = {
    't': [0, 65, 129, 188, 247, 309, 370, 430, 489, 550],
    'h_L': [3.84, 3.76, 3.62, 3.57, 3.45, 3.36, 3.27, 3.15, 3.08, 2.94],
    'h_R': [3.55, 3.68, 3.75, 3.82, 3.90, 4.01, 4.18, 4.26, 4.39, 4.50]
}

df = pd.DataFrame(data)

df

,t,h_L,h_R
0,0,3.84,3.55
1,65,3.76,3.68
2,129,3.62,3.75
3,188,3.57,3.82
4,247,3.45,3.90
5,309,3.36,4.01
6,370,3.27,4.18
7,430,3.15,4.26
8,489,3.08,4.39
9,550,2.94,4.50


In [268]:
X = 0.08

def s_L(t):
    # 添加随机高斯误差
    a_error = np.random.normal(0, X)
    a = 0.00173 * (1 + a_error)
    b_error = np.random.normal(0, 0.008*X)
    return round(a * t + b_error, 2)

def s_R(t):
    # 添加随机高斯误差
    a_error = np.random.normal(0, X)
    a = 0.00168 * (1 + a_error)
    b_error = np.random.normal(0, 0.008*X)
    return round(a * t + b_error, 2)
    


t = [0, 59, 57, 63, 69, 71, 64, 50, 70, 68]
'''data = {
    't': [0, 65, 129, 188, 247, 309, 370, 430, 489, 550]
}'''
# 逐项求和
t = np.array(t).cumsum().tolist()



data = {
    't': t,
    's_L': [s_L(i) for i in t],
    's_R': [s_R(i) for i in t]
}

data = pd.DataFrame(data)

In [269]:
slope_L, intercept_L, r_value_L, p_value_L, std_err_L = stats.linregress(data['t'], data['s_L'])
slope_R, intercept_R, r_value_R, p_value_R, std_err_R = stats.linregress(data['t'], data['s_R'])

# 计算斜率与截距的不确定度
def get_uncertainty(data, slope, intercept):
    n = len(data)
    x = data['t']
    y = data['s_L']
    y_pred = [slope * i + intercept for i in x]
    sigma = np.sqrt(np.sum([(y[i] - y_pred[i]) ** 2 for i in range(n)]) / (n - 2))
    sigma_a = sigma * np.sqrt(np.sum([i ** 2 for i in x]) / (n * np.sum([(i - np.mean(x)) ** 2 for i in x])))
    sigma_b = sigma * np.sqrt(1 / n + np.mean(x) ** 2 / np.sum([(i - np.mean(x)) ** 2 for i in x]))
    return sigma_a, sigma_b

sigma_a_L, sigma_b_L = get_uncertainty(data, slope_L, intercept_L)
sigma_a_R, sigma_b_R = get_uncertainty(data, slope_R, intercept_R)

# 以方程的形式答应结果，并打印R^2
print('s_L = {} * t + {}'.format(round(slope_L, 5), round(intercept_L, 5)), 'R^2 = {}'.format(round(r_value_L ** 2, 5)))
print('s_R = {} * t + {}'.format(round(slope_R, 5), round(intercept_R, 5)), 'R^2 = {}'.format(round(r_value_R ** 2, 5)))

s_L = 0.00179 * t + -0.00197 R^2 = 0.96763
s_R = 0.00168 * t + -0.01275 R^2 = 0.99481


In [270]:
n = len(data['t'])  # 数据点的数量
SSX = np.sum((data['t'] - np.mean(data['t']))**2)  # 总平方和

# 计算截距的不确定度
intercept_err_L = std_err_L * np.sqrt(np.sum(data['t']**2) / (n * SSX))
intercept_err_R = std_err_R * np.sqrt(np.sum(data['t']**2) / (n * SSX))

print(f"Slope L Error: {std_err_L}")
print(f"Intercept L Error: {intercept_err_L}")
print(f"Slope R Error: {std_err_R}")
print(f"Intercept R Error: {intercept_err_R}")

Slope L Error: 0.00011546865435130595
Intercept L Error: 6.689831742145095e-05
Slope R Error: 4.286050105297011e-05
Intercept R Error: 2.4831807561907367e-05


In [292]:
data

,t,s_L,s_R
0,0,0.00,-0.00
1,59,0.11,0.09
2,116,0.21,0.19
3,179,0.32,0.27
4,248,0.47,0.38
5,319,0.47,0.52
6,383,0.67,0.66
7,433,0.89,0.71
8,503,0.80,0.79
9,571,1.06,0.98


In [295]:
# 交换 s_L 与 s_R 的值
data['s_L'], data['s_R'] = data['s_R'], data['s_L']
data

,t,s_L,s_R
0,0,-0.00,0.00
1,59,0.09,0.11
2,116,0.19,0.21
3,179,0.27,0.32
4,248,0.38,0.47
5,319,0.52,0.47
6,383,0.66,0.67
7,433,0.71,0.89
8,503,0.79,0.80
9,571,0.98,1.06


In [300]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter

def format_y(value, tick_number):
    return f'{value:.2f}'

def format_x(value, tick_number):
    return f'{value:.0f}'

label = [r'$\Delta h(-)$', r'$\Delta h(+)$']
labels = ['Fitted Line of ' + _ for _ in label]
colors = ['blue', 'blue', 'red', 'red', 'green', 'green', 'black', 'black']

# 设置 matplotlib 参数
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['mathtext.it'] = 'Times New Roman:italic'
plt.rcParams['mathtext.bf'] = 'Times New Roman:bold'
plt.rcParams['font.size'] = 16
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

# 绘图
plt.figure(figsize=(8, 6), dpi=1200)

plt.scatter(data['t'], data['s_L'], marker='o', color='darkblue', label=label[0])
plt.scatter(data['t'], data['s_R'], marker='s', color='darkred', label=label[1])

plt.plot(data['t'], [slope_L * i + intercept_L for i in data['t']], color='blue', label=labels[0])
plt.plot(data['t'], [slope_R * i + intercept_R for i in data['t']], color='red', label=labels[1])


# 设置标签和图例
plt.xlabel(r'$t$ / s')
plt.ylabel(r'$\Delta h$ / cm')
plt.legend(frameon=False)


# 设置网格线
plt.gca().xaxis.set_major_formatter(FuncFormatter(format_x))
plt.gca().yaxis.set_major_formatter(FuncFormatter(format_y))
plt.gca().xaxis.set_minor_locator(ticker.AutoMinorLocator())
plt.gca().yaxis.set_minor_locator(ticker.AutoMinorLocator())
plt.grid(which='major', alpha=0.5)
plt.grid(which='minor', alpha=0.2)

# 显示图形
plt.tight_layout()
plt.show()

In [301]:
data_df = pd.DataFrame(data)
data_latex = data_df.to_latex('data.tex', index=False)
data_df

,t,s_L,s_R
0,0,-0.00,0.00
1,59,0.09,0.11
2,116,0.19,0.21
3,179,0.27,0.32
4,248,0.38,0.47
5,319,0.52,0.47
6,383,0.66,0.67
7,433,0.71,0.89
8,503,0.79,0.80
9,571,0.98,1.06


In [302]:
s_L_0 = 3.95
s_R_0 = 3.90

# 在data_df的第一列与第二列之间插入一列
data_df.insert(1, '-', data_df['s_L'] + s_L_0)
data_df.insert(3, '+', - data_df['s_R'] + s_R_0)

data_df.to_latex('datadf.tex', index=False)

In [303]:
distance = np.array([22.50, 22.20, 22.75])
dis_std = np.std(distance).round(2)
distance = np.mean(distance).round(2)
distance, dis_std

(22.48, 0.22)

In [304]:
err1 = 0.00004
err2 = 0.00012

err = np.sqrt(err1**2 + err2**2)
err

0.00012649110640673518

In [305]:
%pip install uncertainties

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/98.4 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/98.4 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/98.4 kB ? eta -:--:--
     ----------- -------------------------- 30.7/98.4 kB 163.8 kB/s eta 0:00:01
     ----------- -------------------------- 30.7/98.4 kB 163.8 kB/s eta 0:00:01
     --------------- ---------------------- 41.0/98.4 kB 164.3 kB/s eta 0:00:01
     --------------- ---------------------- 41.0/98.4 kB 164.3 kB/s eta 0:00:01
     --------------- ---------------------- 41.0/98.4 kB 164.3 kB/s eta 0:00:01
     --------------- ---------------------- 41.0/98.4 kB 164.3 kB/s eta 0:00:01
     ----------------------- -------------- 61.4/98.4 kB 131.3 kB/s eta 0:00:01
     -------------------------------------- 98.4/98.4 kB 201.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [314]:
from scipy.constants import epsilon_0
import uncertainties as unc
import uncertainties.unumpy as unp

# 定义给定的参数和它们的不确定性
l = unc.ufloat(22.48, 0.22)  # 电极间距离，单位：cm
v = unc.ufloat(0.00172, 0.00012)  # 界面迁移的平均速度，单位：cm/s
phi = unc.ufloat(100, 1)  # 电势差，单位：V
epsilon_r = unc.ufloat(80.10, 0.03)  # 水的相对介电常数
eta = unc.ufloat(1.083 * 1e-3, 0.04* 1e-3)  # 水的黏度，转换为 Pa·s (1 mPa·s = 1e-3 Pa·s)

# 转换单位：将 l 和 v 从 cm 转换为 m
l_m = l * 1e-2
v_m = v * 1e-2

# 计算电动电势 ζ
zeta = (eta * v_m * l_m) / (phi * epsilon_r * epsilon_0)

# 输出结果
zeta_nominal = unp.nominal_values(zeta)
zeta_std_dev = unp.std_devs(zeta)

zeta_nominal, zeta_std_dev


(array(0.05904348), array(0.00473364))

In [316]:
import pandas as pd

# 已知参数及其不确定性
eta = 1.083e-3  # Pa·s
sigma_eta = 0.004e-3  # Pa·s
v = 0.00172  # m/s
sigma_v = 0.00012  # m/s
l = 0.2248  # m
sigma_l = 0.0022  # m
phi = 100  # V
sigma_phi = 1  # V
epsilon_r = 80.10
sigma_epsilon_r = 0.03

# 计算每个参数的相对误差
relative_errors = {
    "Variable": ["eta", "v", "l", "phi", "epsilon_r"],
    "Value": [eta, v, l, phi, epsilon_r],
    "Standard Deviation": [sigma_eta, sigma_v, sigma_l, sigma_phi, sigma_epsilon_r],
    "Relative Error (%)": [
        (sigma_eta / eta) * 100,
        (sigma_v / v) * 100,
        (sigma_l / l) * 100,
        (sigma_phi / phi) * 100,
        (sigma_epsilon_r / epsilon_r) * 100
    ]
}

# 创建 DataFrame
df_relative_errors = pd.DataFrame(relative_errors)

zeta_error_manual = 0.0047
# 计算 zeta 的相对误差
zeta_relative_error = (zeta_error_manual / zeta_nominal) * 100

# 输出 DataFrame 和 zeta 的相对误差
df_relative_errors, zeta_relative_error



(    Variable       Value  Standard Deviation  Relative Error (%)
 0        eta    0.001083            0.000004            0.369344
 1          v    0.001720            0.000120            6.976744
 2          l    0.224800            0.002200            0.978648
 3        phi  100.000000            1.000000            1.000000
 4  epsilon_r   80.100000            0.030000            0.037453,
 7.9602350810021765)